# Bone Mineral Density Example

This Jupyter Notebook provides an example of computing volumetric bone mineral density (vBMD) in an HR-pQCT joint image. vBMD is computed in the distal and proximal segments. The ORMIR_XCT autocontour script is used to generate the bone masks.

# Step 1: Imports

Import modules/packages and set the input image path. 

In [32]:
import os
import numpy as np
import SimpleITK as sitk

from matplotlib import pyplot as plt

from ormir_xct.bone_mineral_density.bmd import bmd
from ormir_xct.bone_mineral_density.bmd_masked import bmd_masked
from ormir_xct.autocontour.autocontour import autocontour

In [33]:
joint_seg_path = os.path.join("images", "GRAY_JOINT.nii")

gray_img = sitk.ReadImage(joint_seg_path, sitk.sitkFloat32)

# Step 2: Run the ORMIR_XCT Automatic Contour

Run the ORMIR_XCT automatic periosteal contour script on the input grayscale joint image. This script will return the distal, proximal, and full joint mask.

When running the ORMIR_XCT automatic contour script, we need to provide the image units and parameters for unit conversion to get an accurate segmentation. Since we are using an AIM/ISQ image that has been converted to another file type using the ITKIOScanco module from ITK, the image units are Hounsfield Units (HU). For the sample image provided, the follow parameters taken from the AIM header are used:

- mu_water = 0.24090
- mu_scaling = 8192
- resale_slope = 1603.51904 
- rescale_intercept = -391.209015

These values may vary depending on your scanner.

In [34]:
mu_water = 0.24090
mu_scaling = 8192
rescale_slope = 1603.51904 
rescale_intercept = -391.209015

dst_mask, prx_mask, ormir_mask = autocontour(gray_img, mu_water, rescale_slope, rescale_intercept)

# Step 3: Compute vBMD

Now computed vBMD. We will calculate the vBMD for each bone segment.

In [35]:
# First mask the image with the distal and proximal joint masks

# Distal vBMD
masked_gray_img = sitk.Mask(gray_img, dst_mask)
dst_bmd = bmd_masked(masked_gray_img, dst_mask, 'hu', mu_scaling, mu_water, rescale_slope, rescale_intercept)

print('Distal vBMD: {:.2f} +/- {:.2f} mg HA/cm^3'.format(dst_bmd[0], dst_bmd[1]))


# Proximal vBMD
prx_gray_img = sitk.Mask(gray_img, prx_mask)
prx_bmd = bmd_masked(masked_gray_img, prx_mask, 'hu', mu_scaling, mu_water, rescale_slope, rescale_intercept)

print('Proximal vBMD: {:.2f} +/- {:.2f} mg HA/cm^3'.format(prx_bmd[0], prx_bmd[1]))

Distal vBMD: 193.18 +/- 205.73 mg HA/cm^3
Proximal vBMD: 227.12 +/- 240.44 mg HA/cm^3


# Step 4: Compare Between IPL and ORMIR_XCT

Now compare vBMD results between IPL and ORMIR_XCT.

The vBMD results from IPL have been pre-generated and are provided as:
- **Distal vBMD:** 203.84 +/- 212.96 mg HA/cm^3
- **Proximal vBMD:** 224.20 +/- 240.37 mg HA/cm^3